# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

In [2]:
base_dir = r'./data/raw'
folders = os.listdir(base_dir)
print(folders)

['Cloudy', 'Rain', 'Shine', 'Sunrise']


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator

datagenerator = ImageDataGenerator(rescale=1/255, validation_split=0.2)
train_generator = datagenerator.flow_from_directory(
    base_dir,
    target_size=(250 , 250),
    batch_size=32,
    subset='training',
)

test_generator = datagenerator.flow_from_directory(
    base_dir,
    target_size=(250 , 250),
    batch_size=32,
    subset='validation',
)

Found 901 images belonging to 4 classes.
Found 224 images belonging to 4 classes.


In [5]:
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,MaxPooling2D,Conv2D,Dropout

inputs = Input(shape=(250, 250, 3))

x = Conv2D(32, (5,5), input_shape = (250 , 250 , 3), activation = 'relu')(inputs)
x = MaxPooling2D(3,3)(x)
x = Conv2D(64, (5,5) , activation = 'relu')(x)
x = MaxPooling2D((3,3))(x)
x = Conv2D(128 , (5,5) , activation = 'relu')(x)
x = MaxPooling2D(3,3)(x)
x = Flatten()(x)
x = Dropout(0.5, seed = 5)(x)
x = Dense(128 , activation = 'relu')(x)

outputs = Dense(4, activation = 'softmax')(x)

model:Model = Model(inputs=inputs, outputs=outputs)

In [6]:
model.compile(optimizer="Adam", loss="mean_squared_error", metrics=["accuracy"])
model.fit(
    train_generator, 
    epochs = 10,
    steps_per_epoch = train_generator.n // train_generator.batch_size,
    validation_data = test_generator,
    validation_steps = test_generator.n // test_generator.batch_size,
    #callbacks= callbacks
)

Epoch 1/10


: 

: 

In [ ]:
losses = pd.DataFrame(model.history.history)
losses[['loss','val_loss']].plot(title='Model')

In [ ]:
accurracies = pd.DataFrame(model.history.history)
accurracies[['accuracy','val_accuracy']].plot(title='Model')

In [ ]:
y_pred = model.predict(test_generator)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay

print('Accuracy: %.3f' % accuracy_score(Y_test, y_pred), ' -> ', accuracy_score(Y_test, y_pred) * 100, '%')

ConfusionMatrixDisplay.from_predictions(Y_test, y_pred, display_labels=['Group 0', 'Group 1'], cmap='Blues')
plt.show()

In [ ]:
class_names = folders
def predict_one(model):  
    image_batch, classes_batch = next(test_generator)
    predicted_batch = model.predict(image_batch)
    for k in range(0,image_batch.shape[0]):
        image = image_batch[k]
        pred = predicted_batch[k]
        the_pred = np.argmax(pred)
        predicted = class_names[the_pred]
        val_pred = max(pred)
        the_class = np.argmax(classes_batch[k])
        value = class_names[np.argmax(classes_batch[k])]
        plt.figure(k)
        isTrue = (the_pred == the_class)
        plt.title(str(isTrue) + ' - class: ' + value + ' - ' + 'predicted: ' + predicted + '[' + str(val_pred) + ']')
        plt.imshow(image)

predict_one(model)  

In [ ]:
# Hacer confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay
# concatenate n batches

class_names = ['Nublado', 'Lluvia', 'Soleado', 'Amanecer']
n = 10

image_batch, classes_batch = next(test_generator)

for batch in range(n):
    temp = next(test_generator)
    image_batch = np.concatenate((image_batch, temp[0]))
    classes_batch = np.concatenate((classes_batch, temp[1]))

classes_batch = classes_batch.tolist()
#print(classes_batch)
y_predict = model.predict(image_batch)#.reshape(32*(n+1))
#print(y_predict)

y_temp = []
for x in y_predict:
    #print(x)
    # Poner en 1 el máximo y el resto en 0
    the_pred = np.argmax(x)
    x_temp = []
    for i, clase in enumerate(x):
        #print(clase)
        x_temp.append(0 if i != the_pred else 1)
    y_temp.append(x_temp)
    #print(x_temp)
y_predict = y_temp

ConfusionMatrixDisplay.from_predictions(
    y_true = [np.argmax(x) for x in classes_batch],
    #y_true = classes_batch.
    y_pred = [np.argmax(x) for x in y_predict],
    #y_pred = y_predict,
    display_labels=class_names,
    cmap='Blues'
)
plt.savefig('./confusion_matrix.png')                                       
plt.show()